In [15]:
from pymongo import MongoClient

In [16]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [17]:
db, coll = connectCollection('datamad1019','restaurants')

In [18]:
rest = list(coll.find({'name':"Wendy'S"}))

In [19]:
print(len(rest))

41


In [20]:
rest[0]['address']

{'building': '469',
 'coord': [{'$numberDouble': '-73.961704'}, {'$numberDouble': '40.662942'}],
 'street': 'Flatbush Avenue',
 'zipcode': '11225'}

## GeoJSON

{ type: "Point", coordinates: [ 40, 5 ] }

In [21]:
def getLocation(wendy):
    longitude = wendy['address']['coord'][0]['$numberDouble']
    latitude = wendy['address']['coord'][1]['$numberDouble']
    loc = {
        'type':'Point',
        'coordinates':[float(longitude), float(latitude)]
    }
    return loc

In [22]:
getLocation(rest[0])

{'type': 'Point', 'coordinates': [-73.961704, 40.662942]}

In [23]:
for wendy in rest:
    value = {"$set": {'location':getLocation(wendy)}}
    coll.update_one(wendy,value)
    # <collection>.update_one(<document>,<value>)

In [38]:
for wendy in rest:
    value = {"$unset": {'address.coord':''}}
    coll.update_one(wendy,value)
    # <collection>.update_one(<document>,<value>)

In [24]:
rest = list(coll.find({'name':"Wendy'S"}))
rest[0]

{'_id': ObjectId('5dce93c47ee6b896adfa9b6a'),
 'address': {'building': '469',
  'coord': [{'$numberDouble': '-73.961704'}, {'$numberDouble': '40.662942'}],
  'street': 'Flatbush Avenue',
  'zipcode': '11225'},
 'borough': 'Brooklyn',
 'cuisine': 'Hamburgers',
 'grades': [{'date': datetime.datetime(2014, 12, 30, 0, 0),
   'grade': 'A',
   'score': 8},
  {'date': datetime.datetime(2014, 7, 1, 0, 0), 'grade': 'B', 'score': 23},
  {'date': datetime.datetime(2013, 4, 30, 0, 0), 'grade': 'A', 'score': 12},
  {'date': datetime.datetime(2012, 5, 8, 0, 0), 'grade': 'A', 'score': 12}],
 'name': "Wendy'S",
 'restaurant_id': '30112340',
 'location': {'type': 'Point', 'coordinates': [-73.961704, 40.662942]}}

In [33]:
manhattanWendys = coll.find(
    {"location":
     {"$near":
      {"$geometry":
       {"type":"Point",
        "coordinates":[-73.9712,40.7831]
       },
       "$maxDistance":10000
      }
     }
    }
)
manhattanWendys=list(manhattanWendys)

In [34]:
manhattanWendysAnillo = coll.find(
    {"location":
     {"$near":
      {"$geometry":
       {"type":"Point",
        "coordinates":[-73.9712,40.7831]
       },
       "$maxDistance":10000,
       "$minDistance":5000
      }
     }
    }
)
manhattanWendysAnillo=list(manhattanWendysAnillo)

In [35]:
len(manhattanWendys),len(manhattanWendysAnillo)

(14, 9)

In [36]:
import folium

In [72]:
map_city=folium.Map(location=[40.7221,-73.9712], zoom_start=12)
for wendy in manhattanWendys:
    folium.Marker(wendy['location']['coordinates'][::-1],
                    radius=2,
                    icon=folium.Icon(icon='cloud',color='red'), 
                   ).add_to(map_city)
    
for wendy in manhattanWendys[:5]:
    folium.Marker(wendy['location']['coordinates'][::-1],
                    radius=2,
                    icon=folium.Icon(icon='cloud',color='blue'), 
                   ).add_to(map_city)

In [73]:
map_city